# 1. GPU 설정 및 하이퍼파라미터 설정


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
# 현재 파이썬이 실행되고 있는 환경이서 torch 모듈을 이용할 때, GPU를 이용할 수 있는지를 확인
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")
 
# 학습률 설정
learning_rate = 0.001
# 배치사이즈 설정: 파라미터를 업데이트할 때, 계산되는 데이터의 수 / 파라미터는 배치단위로 수행
batch_size = 100
# 클래스의 갯수
num_classes = 10
# 에포크 수
epochs = 5

# 2. 데이터셋 불러오기

In [ ]:
# MNIST 데이터 다운로드 후, Train, Test set 분리;
train_set = torchvision.datasets.MNIST(
    # root: 데이터가 저장될 장소, 경로를 지정
    root = './data/MNIST',
    # train: train 데이터셋인지, test 데이터셋인지 설정
    train = True,
    # download: 인터넷에서 다운로드해 이용할 것인지 결정하는 요소
    download = True,
    # transform: 다운로드할 때, 기본적인 전처리를 동시에 시행할 수 있음
    # transforms.ToTensor(): Tensor 형태로 변환하고, pixel 값을 255로 나누어, 0~1 사이의 숫자로 정규화
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

# 3. DataLoader를 통하여 학습용 데이터 준비하기

In [ ]:
# DataLoader: 다운로드한 dataset을 mini-batch 단위로 분리하여 불러오기 위하 loader
# shuffle: 데이터의 순서를 섞을때 사용, 보통은 train=True, test=False
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

# 4. 모델 설계

In [ ]:
class ConvNet(nn.Module):
      def __init__(self): 
        super(ConvNet, self).__init__()
        
        # in_channels = 1, out_channels = 10, kernel_size = 5, stride = 1인 Conv2d
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # in_channels = 10, out_channels = 20, kernel_size = 5, stride = 1인 Conv2d
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # Dropout 적용 레이어 노드 중 25%에만 dropout을 적용
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        # kernel_size = 2인 Maxpool
        self.mp = nn.MaxPool2d(2)
        # in_features = 320, out_features = 100인 Linear (fully-connected layer)
        self.fc1 = nn.Linear(320,100)
        # in_features = 100, out_features = 10인 Linear (fully-connected layer)
        # 여기서 out_features=10 = num_classes (클래스의 갯수)
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x):
        # conv1 -> maxpool -> relu
        x = F.relu(self.mp(self.conv1(x)))
        # conv2 -> maxpool -> relu 
        x = F.relu(self.mp(self.conv2(x))) 
        # dropout
        x = self.drop2D(x)
        # flatten
        x = x.view(x.size(0), -1)
        # fully-connected layer
        x = self.fc1(x)
        # fully-connected layer
        x = self.fc2(x) 
        return F.log_softmax(x) 

# 5. 모델을 GPU로 설정 및 Criterion, Optimizer 설정

In [ ]:
# 정의한 네트워크를 GPU에 올리는 과정
model = ConvNet().to(device) 
# Cross Entropy 함수 사용
criterion = nn.CrossEntropyLoss().to(device)
# Adam Optimizer 사용
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# 6. 모델 학습

In [ ]:
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        # 모든 모델의 gradient 값을 0으로 설정
        # 이전 배치에서 사용했던 gradient를 현 배치에서는 사용하지 않기 때문에 gradient를 0으로 초기화
        optimizer.zero_grad()
        hypothesis = model(data)
        cost = criterion(hypothesis, target)
        # reqires_grad = Trur인 모든 Parameter Update에 사용되는 Gradient를 계산
        cost.backward()
        # Update 진행
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

# 7. 모델 성능평가

In [ ]:
# dropout과 같이, train과 val/test가 달라지는 레이어를 위하여 사용
model.eval()
# Gradient를 추가 계산하지 않고 고정시킨 상태에서 실행
# Gradient 계산을 위한 메모리를 더 이상 할당할 필요가 없기 때문에 메모리 공간을 차지하지 않도록 미리 선언
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        # 확률이 가장 높은 첫번째 아이템 값을 리턴 
        preds = torch.max(out.data, 1)[1] 
        total += len(target)
        # 전체 TEST Dataset에서 몇 개 맞았는지
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')